In [11]:
import os 
from langchain_google_genai import ChatGoogleGenerativeAI
from soucre_code.question_generator.utils import read_file
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import json
import pandas as pd
import traceback 
import PyPDF2

In [12]:
## in this project i use gemini 

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key="")

In [13]:
RESPONSE_JSON = {
   "1": {
        "saq": "Explain the process of photosynthesis in plants.",
        "expected_answer": {
            "key_points": [
                "Chlorophyll absorbs sunlight",
                "Conversion of carbon dioxide and water",
                "Production of glucose and oxygen"
            ],
            "length": "10-20 words"
        } 
   },
   "2": {
        "saq": "Explain the process of photosynthesis in plants.",
        "expected_answer": {
            "key_points": [
                "Chlorophyll absorbs sunlight",
                "Conversion of carbon dioxide and water",
                "Production of glucose and oxygen"
            ],
            "length": "10-20 words"
        } 
   },
   "3": {
        "saq": "Explain the process of photosynthesis in plants.",
        "expected_answer": {
            "key_points": [
                "Chlorophyll absorbs sunlight",
                "Conversion of carbon dioxide and water",
                "Production of glucose and oxygen"
            ],
            "length": "10-20 words"
        } 
   }
   
}

In [14]:
## this template 1 
TEMPLATE="""
Text:{text}
You are an expert SAQ maker. Given the above text, it is your job to \
create a quiz  of {number} Short Answer Question  for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} SAQ
### RESPONSE_JSON
{response_json}

""" 
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Short Answer Question Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis.
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_SAQ:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [15]:
## create the prompt 

quiz_generation_prompt=PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
) 

## now create a chain for prompt one 

llm_chain1=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz")

In [16]:
## create prompt for second template 

quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
) 

review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review") 

In [17]:
## now use squentail chain 

generate_evaluate_chain_saq=SequentialChain(chains=[llm_chain1, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"])

In [18]:
## now test our model 
file_path="/home/sahil/Question_Answ/data.txt"
# with open(file_path, 'r') as file:
#     TEXT = file.read()
TEXT=read_file(file_path)

AttributeError: 'str' object has no attribute 'name'

In [9]:
NUMBER=4
SUBJECT="Mechin learning"
TONE="simple"

In [10]:
response=generate_evaluate_chain_saq(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

/home/sahil/Question_Answ/question/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


In [64]:
quiz=response.get("quiz")
# response

In [65]:
quiz=quiz.split("### RESPONSE_JSON\n")[1] 
# quiz.split('```json\n')
quiz

'{"1": {"saq": "What is the difference between supervised and unsupervised learning in machine learning?", "expected_answer": {"key_points": ["Supervised learning uses labeled data to train models, while unsupervised learning uses unlabeled data to find patterns.", "Supervised learning focuses on prediction, while unsupervised learning focuses on discovering hidden structures in data."], "length": "20-30 words"}}, "2": {"saq": "What are the two main objectives of modern-day machine learning?", "expected_answer": {"key_points": ["Classifying data based on developed models.", "Making predictions for future outcomes based on these models."], "length": "10-20 words"}}, "3": {"saq": "What is the main difference between machine learning and statistics?", "expected_answer": {"key_points": ["Statistics draws population inferences from a sample, while machine learning finds generalizable predictive patterns."], "length": "10-20 words"}}, "4": {"saq": "What is feature learning in machine learnin

In [66]:
quiz=json.loads(quiz)

In [16]:
llm_chain1

LLMChain(prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert SAQ maker. Given the above text, it is your job to create a quiz  of {number} Short Answer Question  for {subject} students in {tone} tone.\nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} SAQ\n### RESPONSE_JSON\n{response_json}\n\n'), llm=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x79ae40d22950>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x79ae3652e710>, default_metadata=()), output_key='quiz')

In [56]:
# quiz_table_data = []
# for key, value in quiz.items():
#     mcq = value["saq"]
#     options = " | ".join(
#         [
#             f"{option}: {option_value}"
#             for option, option_value in value["options"].items()
#             ]
#         )
#     correct = value["correct"]
#     quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct}) 
quiz

{'1': {'saq': 'What is the difference between supervised learning and unsupervised learning in machine learning?',
  'expected_answer': {'key_points': ['Supervised learning uses labeled data, unsupervised learning uses unlabeled data'],
   'length': '10-20 words'}},
 '2': {'saq': 'What are the two main objectives of modern-day machine learning?',
  'expected_answer': {'key_points': ['Classifying data based on developed models',
    'Making predictions for future outcomes'],
   'length': '10-20 words'}},
 '3': {'saq': "What is the concept of 'generalization' in machine learning, and why is it important?",
  'expected_answer': {'key_points': ['Ability of a learning machine to perform accurately on new, unseen examples',
    'Important for the model to be applicable to real-world scenarios'],
   'length': '20-30 words'}},
 '4': {'saq': 'What is the key difference between machine learning and data mining, despite their overlapping methods?',
  'expected_answer': {'key_points': ['Machine le

In [70]:
quiz_table_data = []
for key, value in quiz.items():
    saq=value["saq"] 
    
    k=(value["expected_answer"]["key_points"])
    quiz_table_data.append({"SAQ": saq,  "Answer": k})

What is the difference between supervised and unsupervised learning in machine learning?

What are the two main objectives of modern-day machine learning?

What is the main difference between machine learning and statistics?

What is feature learning in machine learning, and what are its advantages?



In [69]:
quiz_table_data

[{'MCQ': 'What is the difference between supervised and unsupervised learning in machine learning?',
  'Correct': ['Supervised learning uses labeled data to train models, while unsupervised learning uses unlabeled data to find patterns.',
   'Supervised learning focuses on prediction, while unsupervised learning focuses on discovering hidden structures in data.']},
 {'MCQ': 'What are the two main objectives of modern-day machine learning?',
  'Correct': ['Classifying data based on developed models.',
   'Making predictions for future outcomes based on these models.']},
 {'MCQ': 'What is the main difference between machine learning and statistics?',
  'Correct': ['Statistics draws population inferences from a sample, while machine learning finds generalizable predictive patterns.']},
 {'MCQ': 'What is feature learning in machine learning, and what are its advantages?',
  'Correct': ['Feature learning discovers better representations of input data, often as a pre-processing step.',
   'I

In [71]:
quiz=pd.DataFrame(quiz_table_data)

In [72]:
quiz.to_csv("ml.csv",index=False)